In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [20]:
#load data
service_tickets = pd.read_csv('../data/bi_case_calls_combined.csv')
payments = pd.read_csv('../data/bi_case_payments_combined.csv')
calls = pd.read_csv('../data/bi_case_calls_combined.csv')
contracts = pd.read_csv('../cleaned_data/cleaned_contracts.csv')

In [21]:
payments.head()

,contract_id,pay_month,total_paid
0,2685526,2024-01-31,49.4997
1,2685526,2024-02-29,25.0834
2,2685528,2024-02-29,9.5287
3,2685528,2024-03-31,3.0738
4,2685546,2024-01-31,NaN


In [22]:
# number of rows
len(payments)

1729777

In [23]:
payments.dtypes

contract_id      int64
pay_month       object
total_paid     float64
dtype: object

In [24]:
payments.isna().sum()

contract_id        0
pay_month          0
total_paid     14407
dtype: int64

In [25]:
# work with sql in my notebook to have all my work in the same place
import sqlite3
conn = sqlite3.connect('dlight.db')


In [26]:
#make connection
cursor = conn.cursor()


In [77]:
## create databases
contracts.to_sql('cleaned_contracts', conn, index=False, if_exists='replace')

payments.to_sql('payments',conn, index=False, if_exists='replace')

service_tickets.to_sql('service_tickets', conn, index=False, if_exists='replace')

calls.to_sql('calls', conn, index=False, if_exists='replace')


134023

In [84]:
# test connection
df = pd.read_sql_query("SELECT * FROM cleaned_contracts LIMIT 5;", conn)
df


,contractid,sales_month,region,product,sales_person_id,contract_type,price_usd,payment_frequency,perc_deposit,tenor_length,daily_amount_usd,customer_gender,household_size,occupation,actual_household_size,phone_number,name,assumption_flag,incomplete_flag
0,3098268,2024-10-31,Western,Small Solar,154077.0,FINANCED,150.0,WEEKLY,0.0510,434.0,0.32800,Male,None,Teacher,NaN,None,None,ORIGINAL,0
1,3554125,2025-04-30,Western,Small Solar,878345.0,FINANCED,150.0,WEEKLY,0.0313,434.0,0.33500,Male,None,Teacher,NaN,None,None,ORIGINAL,0
2,2792627,2024-03-31,Western,Small Solar,648775.0,FINANCED,150.0,WEEKLY,0.0554,434.0,0.32600,Male,4,Government Employee,4.0,None,None,ORIGINAL,0
3,2869807,2024-05-31,Western,PAYGO_PHONE,328066.0,FINANCED,200.0,WEEKLY,0.1827,189.0,0.86500,Male,None,Business,NaN,None,None,ORIGINAL,0
4,3674681,2025-06-30,Western,Small Solar,813534.0,FINANCED,150.0,WEEKLY,0.0296,364.0,0.39989,Male,None,Business,NaN,None,None,ASSUMPTION,0


Investigate null values

In [ ]:
total_null = """SELECT c.contractid, c.contract_type, c.sales_month, p.total_paid,p.pay_month,
                c.price_usd,c.product,c.payment_frequency,c.perc_deposit,c.tenor_length,c.daily_amount_usd
                FROM cleaned_contracts c
                JOIN payments p ON
                c.contractid = p.contract_id
                WHERE total_paid IS NOT NULL
                AND daily_amount_usd IS NOT NULL
                AND contractid = '3098268'
"""

pd.read_sql(total_null, conn)

,contractid,contract_type,sales_month,total_paid,pay_month,price_usd,product,payment_frequency,perc_deposit,tenor_length,daily_amount_usd
0,3098268,FINANCED,2024-10-31,25.5102,2024-10-31 00:00:00,150.0,Small Solar,WEEKLY,0.051,434.0,0.328
1,3098268,FINANCED,2024-10-31,25.5102,2024-11-30 00:00:00,150.0,Small Solar,WEEKLY,0.051,434.0,0.328
2,3098268,FINANCED,2024-10-31,12.7551,2025-01-31 00:00:00,150.0,Small Solar,WEEKLY,0.051,434.0,0.328


- despite having the same contract terms, we see total amount paid varying for some contracts. 

-  total paid appears to be transactional data, it reflects irregular payments, partial payments and missed payments. total paid cannot be derieved from contract data

In [85]:

# Aggregate total revenue per product
revenue_summary = contracts.groupby('product')['price_usd'].sum().reset_index(name='total_expected_revenue')

# Sort by total revenue descending
revenue_summary = revenue_summary.sort_values(by='total_expected_revenue', ascending=False)

print(revenue_summary)
"""small solar is expected to generate the highest revenue"""


                      product  total_expected_revenue
4                 Small Solar              29737200.0
2                 PAYGO_PHONE              23616600.0
0  Large Solar - Generation 1              22870800.0
1  Large Solar - Generation 2              14587720.0
3              PAYGO_PORTABLE               3979100.0


'small solar is expected to generate the highest revenue'

In [ ]:
"""compare contract value with revenue collected"""

total_revenue = """
SELECT 
    c.product,
    SUM(c.price_usd) as total_contract_value,
    COALESCE(SUM(p.total_payments), 0) as total_revenue_collected
FROM cleaned_contracts c
LEFT JOIN (
    SELECT contract_id, SUM(total_paid) as total_payments
    FROM payments
    GROUP BY contract_id
) p ON c.contractid = p.contract_id
WHERE product IS NOT NULL
GROUP BY c.product
"""
pd.read_sql(total_revenue, conn)

,product,total_contract_value,total_revenue_collected
0,Large Solar - Generation 1,22870800.0,1.296216e+10
1,Large Solar - Generation 2,14587720.0,3.280370e+06
2,PAYGO_PHONE,23616600.0,6.646863e+06
3,PAYGO_PORTABLE,3979100.0,2.100721e+07
4,Small Solar,29737200.0,3.620388e+10


Total revenue collected seems to be way higher than the contract value

In [44]:
# Check for duplicate records
"""there are no duplicate records"""

check_duplicates = """
SELECT 
    contract_id,
    pay_month,
    total_paid,
    COUNT(*) as duplicate_count
FROM payments
GROUP BY contract_id, pay_month, total_paid
HAVING COUNT(*) > 1
LIMIT 20
"""
pd.read_sql(check_duplicates, conn)

,contract_id,pay_month,total_paid,duplicate_count


In [45]:
# Check total payments in the payments table

payment_records = """
SELECT 
    COUNT(DISTINCT contract_id) as contracts_with_payments,
    SUM(total_paid) as total_payments
FROM payments
"""
pd.read_sql(payment_records, conn)

,contracts_with_payments,total_payments
0,486154,4.919698e+10


In [ ]:
# How many total contracts have been initiated
all_contracts = """
SELECT 
    COUNT(DISTINCT contractid) as total_contracts_ever,
    MIN(sales_month) as earliest_contract,
    MAX(sales_month) as latest_contract
FROM cleaned_contracts
"""
pd.read_sql(all_contracts, conn)

,total_contracts_ever,earliest_contract,latest_contract
0,486623,2024-01-31,2025-09-30


In [ ]:
# total value of ALL contracts ever sold
contract_value = """
SELECT 
    SUM(price_usd) as total_contract_all_time_value,
    COUNT(contractid) as total_contracts
FROM cleaned_contracts
"""
pd.read_sql(contract_value, conn)

,total_contract_all_time_value,total_contracts
0,94791420.0,486623


## Outlier Analysis

In [ ]:
# payment distribution
payment_distribution = """
SELECT 
    COUNT(*) as total_payments,
    MIN(total_paid) as min_payment,
    MAX(total_paid) as max_payment,
    AVG(total_paid) as avg_payment,
    SUM(total_paid) as sum_payments
FROM payments
WHERE contract_id IN (
    SELECT contractid 
    FROM cleaned_contracts 
)
AND total_paid IS NOT NULL
"""
pd.read_sql(payment_distribution, conn)



,total_payments,min_payment,max_payment,avg_payment,sum_payments
0,1715370,0.0,7.587000e+09,28680.100428,4.919698e+10


As we can see we have a maximum payment of 7.5 billion which should not be the case

In [ ]:
# Count outliers
outlier_summary = """
SELECT 
    COUNT(*) as num_outliers,
    SUM(p.total_paid) as total_outlier_value,
    AVG(p.total_paid) as avg_outlier_value,
    (SELECT max(price_usd) FROM contracts) as threshold_used
    FROM payments p
    INNER JOIN contracts c 
    ON p.contract_id = c.contractid
    WHERE
    p.total_paid IS NOT NULL
    AND p.total_paid > (SELECT MAX(price_usd) FROM cleaned_contracts)
"""
pd.read_sql(outlier_summary,conn)

,num_outliers,total_outlier_value,avg_outlier_value,threshold_used
0,761,4.917286e+10,6.461611e+07,300.0


In [ ]:
# Outliers by products
outliers_by_product = """
SELECT 
    c.product,
    COUNT(*) as num_outlier_payments,
    SUM(p.total_paid) as total_outlier_value,
    AVG(p.total_paid) as avg_outlier_value,
    MIN(p.total_paid) as min_outlier,
    MAX(p.total_paid) as max_outlier
FROM payments p
INNER JOIN contracts c ON p.contract_id = c.contractid
WHERE
    p.total_paid IS NOT NULL
    AND p.total_paid > (SELECT MAX(price_usd) FROM cleaned_contracts)
GROUP BY c.product
ORDER BY total_outlier_value DESC
"""
pd.read_sql(outliers_by_product, conn)



,product,num_outlier_payments,total_outlier_value,avg_outlier_value,min_outlier,max_outlier
0,Small Solar,34,3.619541e+10,1.064571e+09,3.109459e+02,1.867500e+09
1,Large Solar - Generation 1,687,1.295730e+10,1.886070e+07,3.000003e+02,7.587000e+09
2,PAYGO_PORTABLE,1,2.000000e+07,2.000000e+07,2.000000e+07,2.000000e+07
3,PAYGO_PHONE,1,1.300000e+05,1.300000e+05,1.300000e+05,1.300000e+05
4,Large Solar - Generation 2,38,1.201321e+04,3.161372e+02,3.112849e+02,3.927000e+02


In [ ]:
outliers_by_month = """
SELECT 
    p.pay_month,
    COUNT(*) as num_outlier_payments,
    SUM(p.total_paid) as total_outlier_value,
    AVG(p.total_paid) as avg_outlier_value
FROM payments p
INNER JOIN contracts c ON p.contract_id = c.contractid
WHERE 
    p.total_paid IS NOT NULL
    AND p.total_paid > (SELECT MAX(price_usd) FROM cleaned_contracts)
GROUP BY p.pay_month
ORDER BY p.pay_month
"""
print(pd.read_sql(outliers_by_month, conn))

     pay_month  num_outlier_payments  total_outlier_value  avg_outlier_value
0   2023-01-31                     1         4.500000e+02       4.500000e+02
1   2024-01-31                    12         3.754159e+03       3.128466e+02
2   2024-02-29                    63         1.921211e+04       3.049542e+02
3   2024-03-31                    51         1.560216e+04       3.059248e+02
4   2024-04-30                    68         2.056952e+04       3.024929e+02
5   2024-05-31                    77         2.500219e+04       3.247038e+02
6   2024-06-30                    38         1.581751e+10       4.162502e+08
7   2024-07-31                    48         1.252514e+09       2.609405e+07
8   2024-08-31                    36         1.867511e+09       5.187529e+07
9   2024-09-30                    33         1.030505e+04       3.122743e+02
10  2024-10-31                    25         7.674469e+03       3.069788e+02
11  2024-11-30                    43         1.364094e+04       3.172311e+02

In [ ]:
#Outliers by contract
contract_outliers = """
SELECT c.contractid, c.product, c.price_usd, SUM(p.total_paid) as revenue_collected
FROM cleaned_contracts c
LEFT JOIN payments p ON c.contractid = p.contract_id
GROUP BY c.contractid, c.product, c.price_usd
ORDER BY revenue_collected DESC"""

pd.read_sql(contract_outliers, conn)


,contractid,product,price_usd,revenue_collected
0,3503595,Large Solar - Generation 1,300.0,7.587000e+09
1,3468141,Large Solar - Generation 1,300.0,5.370000e+09
2,2927799,Small Solar,150.0,1.867500e+09
3,2927808,Small Solar,150.0,1.867500e+09
4,2927818,Small Solar,150.0,1.867500e+09
...,...,...,...,...
486618,3820221,Small Solar,150.0,NaN
486619,3820398,Small Solar,150.0,NaN
486620,3820450,None,NaN,NaN
486621,3820618,None,NaN,NaN


## Data Quality Issue: Payment Outliers

### Problem Identified
We discovered 761 outlier payment records (0.04% of all payments) that inflated reported revenue by 61,210%.


### Why These Are Data Errors

1. **Economic Impossibility**: Customers paying billions for $150-$300 products
2. **Business Model Mismatch**: PAYG customers make small daily payments ($0.30/day), not billion-dollar transfers
3. **Mathematical Impossibility**: At normal payment rates, these amounts would take millions of years to accumulate
4. **Concentrated Pattern**: Outliers cluster in specific months (May 2024, Dec 2024), indicating system errors rather than legitimate payments

### Impact

- **Before Cleaning**: Total revenue = $49.2 billion, Collection rate = 61,210%
- **After Cleaning**: Total revenue = $24.1 million, Collection rate = 25.29%

### Root Cause
Most likely causes:
- Currency conversion error (multiplied instead of divided)
- Database corruption during data migration
- Decimal point misplacement
- Software bug in payment processing system

### Resolution
Created two datasets:
- `payments_cleaned`
- `outlier_payments`

In [ ]:
# Step 1: check for outliers
outlier_check = """SELECT 
    COUNT(*) as outlier_rows,
    SUM(p.total_paid) as outlier_value,
    (SELECT MAX(price_usd) FROM contracts) as threshold_used
FROM payments p
WHERE p.total_paid IS NOT NULL
AND p.total_paid > (SELECT MAX(price_usd) FROM cleaned_contracts)"""

outliers = pd.read_sql(outlier_check, conn)
print(outliers)
print(f"\nWill remove {outliers['outlier_rows'].iloc[0]:,} payment rows")
print(f"Threshold: ${outliers['threshold_used'].iloc[0]:,}")


   outlier_rows  outlier_value  threshold_used
0           761   4.917286e+10           300.0

Will remove 761 payment rows
Threshold: $300.0


In [ ]:
# Step 2: Create the cleaned payments table
cleaned = """
CREATE TABLE cleaned_payments AS
WITH max_price AS (
    SELECT MAX(price_usd) as threshold
    FROM cleaned_contracts
)
SELECT p.*
FROM payments p
CROSS JOIN max_price mp
WHERE p.total_paid IS NOT NULL
    AND p.total_paid <= mp.threshold
"""

conn.executescript(cleaned)
conn.commit()
print("Created 'cleaned_payments' table")

In [70]:
# Step 3: Verify the cleaned table
verification = """
SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT contract_id) as unique_contracts,
    MIN(pay_month) as earliest_payment,
    MAX(pay_month) as latest_payment,
    MIN(total_paid) as min_payment,
    MAX(total_paid) as max_payment,
    ROUND(AVG(total_paid), 2) as avg_payment,
    ROUND(SUM(total_paid), 2) as total_value
FROM cleaned_payments
"""
pd.read_sql(verification, conn)

,total_rows,unique_contracts,earliest_payment,latest_payment,min_payment,max_payment,avg_payment,total_value
0,1714609,480851,2021-01-31,2025-12-31,0.0,300.0,14.07,24127042.15


In [74]:
df = pd.read_sql("SELECT * FROM cleaned_payments", conn)
df.to_csv('../cleaned_data/cleaned_payments.csv', index=False)
print(f" Exported {len(df):,}")

 Exported 1,714,609


In [75]:
outliers = """CREATE TABLE outliers_payments AS
WITH max_price AS (
    SELECT MAX(price_usd) as threshold
    FROM contracts
)
SELECT p.*
FROM payments p
CROSS JOIN max_price mp
WHERE p.total_paid IS NOT NULL
    AND p.total_paid > mp.threshold"""

conn.execute(outliers)
conn.commit()

To ensure a fair analysis, we will compare its performance only over the period since its launch,
aligning other products to the same timeframe for an accurate benchmark.


In [80]:
# Large Solar - Generation 2 was launched later than the other products.
contracts.groupby('product')['sales_month'].agg(['min', 'max'])


,min,max
product,,
Large Solar - Generation 1,2024-01-31,2025-09-30
Large Solar - Generation 2,2024-06-30,2025-09-30
PAYGO_PHONE,2024-01-31,2025-09-30
PAYGO_PORTABLE,2024-01-31,2025-09-30
Small Solar,2024-01-31,2025-09-30


In [82]:
# filter all our contract data to when the 2nd gen was launched to have a fair analysis of our data
filter_dates= """
SELECT *
FROM cleaned_contracts
WHERE sales_month >= '2024-06-30'
"""
# Execute the query
pd.read_sql(filter_dates,conn)


,contractid,sales_month,region,product,sales_person_id,contract_type,price_usd,payment_frequency,perc_deposit,tenor_length,daily_amount_usd,customer_gender,household_size,occupation,actual_household_size,phone_number,name,assumption_flag,incomplete_flag
0,3098268,2024-10-31,Western,Small Solar,154077.0,FINANCED,150.0,WEEKLY,0.0510,434.0,0.32800,Male,None,Teacher,NaN,NaN,None,ORIGINAL,0
1,3554125,2025-04-30,Western,Small Solar,878345.0,FINANCED,150.0,WEEKLY,0.0313,434.0,0.33500,Male,None,Teacher,NaN,NaN,None,ORIGINAL,0
2,3674681,2025-06-30,Western,Small Solar,813534.0,FINANCED,150.0,WEEKLY,0.0296,364.0,0.39989,Male,None,Business,NaN,NaN,None,ASSUMPTION,0
3,3743268,2025-08-31,Southern,Large Solar - Generation 2,195020.0,FINANCED,280.0,DAILY,0.0442,540.0,0.49600,Male,None,Labourer,NaN,NaN,None,ORIGINAL,0
4,3311105,2025-01-31,Southern,Large Solar - Generation 2,974482.0,FINANCED,280.0,DAILY,0.0442,540.0,0.49600,Female,None,Business,NaN,NaN,None,ORIGINAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415195,3817296,2025-09-30,Southern,Small Solar,670507.0,FINANCED,150.0,DAILY,0.0495,480.0,0.29700,Male,None,Labourer,NaN,NaN,None,ORIGINAL,0
415196,3817534,2025-09-30,Southern,Small Solar,884340.0,FINANCED,150.0,DAILY,0.0495,480.0,0.29700,Female,None,Business,NaN,NaN,None,ORIGINAL,0
415197,3817434,2025-09-30,Northern,PAYGO_PORTABLE,246897.0,FINANCED,100.0,TRIDAILY,0.0401,399.0,0.24100,Male,None,Farmer,NaN,NaN,None,ORIGINAL,0
415198,3817380,2025-09-30,Northern,Large Solar - Generation 2,500619.0,FINANCED,280.0,DAILY,0.0565,640.0,0.41300,Female,None,Business,NaN,NaN,None,ORIGINAL,0


In [89]:
# filter all our contract and payment data to when the 2nd gen was launched to have a fair analysis of our data
filter_dates = """
SELECT c.*, p.*
FROM cleaned_contracts c
LEFT JOIN payments p
  ON c.contractid = p.contract_id
WHERE c.sales_month >= '2024-06-30'
"""
pd.read_sql(filter_dates, conn)


,contractid,sales_month,region,product,sales_person_id,contract_type,price_usd,payment_frequency,perc_deposit,tenor_length,...,household_size,occupation,actual_household_size,phone_number,name,assumption_flag,incomplete_flag,contract_id,pay_month,total_paid
0,3098268,2024-10-31,Western,Small Solar,154077.0,FINANCED,150.0,WEEKLY,0.0510,434.0,...,None,Teacher,NaN,NaN,None,ORIGINAL,0,3098268.0,2024-10-31,25.5102
1,3098268,2024-10-31,Western,Small Solar,154077.0,FINANCED,150.0,WEEKLY,0.0510,434.0,...,None,Teacher,NaN,NaN,None,ORIGINAL,0,3098268.0,2024-11-30,25.5102
2,3098268,2024-10-31,Western,Small Solar,154077.0,FINANCED,150.0,WEEKLY,0.0510,434.0,...,None,Teacher,NaN,NaN,None,ORIGINAL,0,3098268.0,2025-01-31,12.7551
3,3554125,2025-04-30,Western,Small Solar,878345.0,FINANCED,150.0,WEEKLY,0.0313,434.0,...,None,Teacher,NaN,NaN,None,ORIGINAL,0,3554125.0,2025-04-30,14.0625
4,3554125,2025-04-30,Western,Small Solar,878345.0,FINANCED,150.0,WEEKLY,0.0313,434.0,...,None,Teacher,NaN,NaN,None,ORIGINAL,0,3554125.0,2025-06-30,4.6875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480245,3817380,2025-09-30,Northern,Large Solar - Generation 2,500619.0,FINANCED,280.0,DAILY,0.0565,640.0,...,None,Business,NaN,NaN,None,ORIGINAL,0,3817380.0,2025-12-31,6.4670
1480246,3820831,2025-09-30,Western,Small Solar,153455.0,FINANCED,150.0,WEEKLY,0.0268,406.0,...,None,Business,NaN,NaN,None,ORIGINAL,0,3820831.0,2025-09-30,10.0671
1480247,3820831,2025-09-30,Western,Small Solar,153455.0,FINANCED,150.0,WEEKLY,0.0268,406.0,...,None,Business,NaN,NaN,None,ORIGINAL,0,3820831.0,2025-10-31,15.1007
1480248,3820831,2025-09-30,Western,Small Solar,153455.0,FINANCED,150.0,WEEKLY,0.0268,406.0,...,None,Business,NaN,NaN,None,ORIGINAL,0,3820831.0,2025-11-30,5.0336


In [90]:
# save the filtered data into a different table
gen2_analysis= pd.read_sql(filter_dates, conn)
gen2_analysis.to_sql('gen2_analysis', conn, index=False, if_exists='replace')


1480250

In [91]:
# List all tables
tables_query = """
SELECT name, type 
FROM sqlite_master 
WHERE type='table'
ORDER BY name
"""

tables = pd.read_sql(tables_query, conn)
print(tables)

                     name   type
0                   calls  table
1       cleaned_contracts  table
2        cleaned_payments  table
3           gen2_analysis  table
4  null_financed_products  table
5     null_price_products  table
6       outliers_payments  table
7                payments  table
8         service_tickets  table


In [ ]:
pd.read_csv('../cleaned_data/cleaned_payments.csv')

,contract_id,pay_month,total_paid
0,2685526,2024-01-31,49.4997
1,2685526,2024-02-29,25.0834
2,2685528,2024-02-29,9.5287
3,2685528,2024-03-31,3.0738
4,2685550,2024-01-31,76.0870
...,...,...,...
1714604,3820655,2025-10-31,9.9125
1714605,3820655,2025-11-30,11.6896
1714606,3820669,2025-12-31,7.8429
1714607,3820684,2025-11-30,6.3638
